In [1]:
from langchain import hub
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings.johnsnowlabs import JohnSnowLabsEmbeddings


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    callback_manager=callback_manager,
    endpoint="http://127.0.0.1:8001/completion",
)

#n_gpu_layers = 1
#n_batch = 32

#llm = LlamaCpp(
#    model_path="/models/Meta-Llama-3-8B-Instruct.Q5_0.gguf",
#    n_gpu_layers=n_gpu_layers,
#    n_batch=n_batch,
#    callback_manager=callback_manager,
#    n_ctx=1024,
#    verbose=True,
#)

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: WARNING! endpoint is not default parameter.
                endpoint was transferred to model_kwargs.
                Please confirm that endpoint is what you intended.
  if await self.run_code(code, result, async_=asy):


KeyError: 'model_path'

In [6]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
import requests

class LlamaCppAPI(LLM):
    endpoint: str = "http://localhost:8001/completion"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {"Content-Type": "application/json"}
        data = {
            "prompt": prompt,
            "n_predict": 128,
        }
        response = requests.post(self.endpoint, json=data, headers=headers)
        return response.json().get("choices", [{}])[0].get("text", "").strip()

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"endpoint": self.endpoint}

    @property
    def _llm_type(self) -> str:
        return "llama-cpp-api"

# 建立 LLM 實例
llm = LlamaCppAPI(endpoint="http://localhost:8001/completion")

# 測試 API 互動
response = llm("請解釋黑洞的概念")
print(response)


ConnectionError: HTTPConnectionPool(host='localhost', port=8001): Max retries exceeded with url: /completion (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f735910ea90>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [3]:
import requests

res = requests.get("http://llamacpp:8001/v1/models")
print(res.json())

{'object': 'list', 'data': [{'id': '/models/DeepSeek-R1-Distill-Llama-8B-Q8_0.gguf', 'object': 'model', 'created': 1742784418, 'owned_by': 'llamacpp', 'meta': {'vocab_type': 2, 'n_vocab': 128256, 'n_ctx_train': 131072, 'n_embd': 4096, 'n_params': 8030261312, 'size': 8532934912}}]}


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# llama.cpp server 模擬 OpenAI API，所以可以用 ChatOpenAI 接
llm = ChatOpenAI(
    base_url="http://llamacpp:8001/v1",  # 注意！container 名稱就是 host 名稱
    api_key="not-needed",  # 這邊寫什麼都行，llama.cpp 不檢查
)

# 測試一下
response = llm([
    HumanMessage(content="請用繁體中文介紹一下你自己")
])
print(response.content)

<think>
您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。
</think>

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


In [5]:
pdf = "Abbass-Dick-2020-32739716.pdf"

loader = PyPDFLoader("/data/" + pdf)
pages = loader.load_and_split()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, add_start_index=True)
all_splits = text_splitter.split_documents(pages)

print(all_splits[0].page_content)

Midwifery 90 (2020) 102812 
Contents lists available at ScienceDirect 
Midwifery 
journal homepage: www.elsevier.com/locate/midw 
The comparison of access to an eHealth resource to current practice 
on mother and co-parent teamwork and breastfeeding rates: A 
randomize d controlle d trial 
Jennifer Abbass-Dick a , ∗, Winnie Sun a , Amber Newport b , Fangli Xie b , Debbie Godfrey b , 
William M. Goodman c


In [6]:
question = "What are the outcomes in this study?"

vectorstore = FAISS.from_documents(all_splits, JohnSnowLabsEmbeddings())


retriever = vectorstore.as_retriever(search_type="similarity")
retrieved_docs = retriever.invoke(question)
print(retrieved_docs[0].page_content)

🤓 Looks like /home/user/.johnsnowlabs is missing, creating it
👷 Setting up  John Snow Labs home in /home/user/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /home/user/.johnsnowlabs


25/03/24 02:51:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.5.2, running on ⚡ PySpark==3.4.0


25/03/24 02:52:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[ | ]

25/03/24 02:52:14 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/03/24 02:52:14 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


sent_small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
Download done! Loading the resource.
[ / ]

2025-03-24 02:52:28.586887: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


Dick et al., 2017 ; Reyes et al. 2019 ; Susin et al., 1999 ). 
The use of breastfeeding educational resources other than the 
eHealth resource provided to SC1 was assessed. Both groups used 
additional resources at all time points, with the greatest use be-


In [7]:
rag_prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)


rag_chain.invoke(question)

"<think>\nOkay, so I need to figure out the outcomes of this study based on the provided context. Let me start by reading through the context carefully. \n\nThe context mentions several studies by Dick et al. (2017), Reyes et al. (2019), and Susin et al. (1999). The study in question is about breastfeeding educational resources, specifically comparing an eHealth resource provided to SC1 against other resources used by both groups. \n\nLooking at the results, it says that breastfeeding rates were high in both groups. For exclusive breastfeeding at 6 months, SC1 had 63% and SC2 had 57%. At 12 months, SC1 was at 71% and SC2 at 78%. These percentages are quite high, and the text notes that these rates are not statistically significantly different between the groups. \n\nAdditionally, the secondary outcomes, which include breastfeeding self-efficacy for mothers and co-parents, showed high scores in both groups. Breastfeeding self-efficacy was measured on a 5-point scale, with higher scores 

In [ ]:
llm.generate(['Do you know what is srdrplus? It\'s maybe related to EBM'])

CUDA error: an illegal memory access was encountered
  current device: 0, in function ggml_cuda_op_mul_mat at /tmp/pip-install-0a91mrof/llama-cpp-python_9508dc37affc43bb8607e7929c051e7c/vendor/llama.cpp/ggml/src/ggml-cuda/ggml-cuda.cu:1613
  cudaGetLastError()
